In [1]:
import numpy as np
import numpy.typing as npt
import gym

from typing import List, Tuple, Literal, Any, Optional, cast, Callable, Union
import plotly.graph_objects as go
from utils.agent import Agent
from tqdm.autonotebook import tqdm
from utils.algorithm import AlgorithmInterface
from utils.preprocess import PreprocessInterface
import torch
from collections import deque
from torchvision import transforms
import math
from torch import nn
import cv2
from PIL import Image
from matplotlib import pyplot as plt
from gym.wrappers import FrameStack
from gym.spaces import Box
import sys
from torchvision import transforms as T
from copy import deepcopy
from utils.common import Step, Episode, TransitionGeneric
from gym.utils.play import play


In [2]:
RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [4]:
env = gym.make("BoxingDeterministic-v4")
# env = gym.make("BoxingDeterministic-v4", obs_type="rgb")
env.seed(RANDOM_SEED)
TOTAL_ACTIONS = env.action_space.n


A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


In [5]:
o = env.reset()
(o.shape, o.dtype)

((210, 160, 3), dtype('uint8'))

In [6]:
env.observation_space.low

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [7]:
env.reward_range

(-inf, inf)

In [8]:
TOTAL_ACTIONS

18

In [9]:
class SkipFrame(gym.ObservationWrapper):
    def __init__(self, env: gym.Env, skip: int):
        assert skip >= 0

        """Return only every `skip`-th frame"""
        super().__init__(env)

        self.env = env
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = False
        obs = None
        info = None

        for _ in range(self._skip + 1):
            # Accumulate reward and repeat the same action
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info


class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self, env: gym.Env):
        super().__init__(env)

        (_, h, w) = env.observation_space.shape
        self.observation_space = Box(
            low=-float("inf"),
            high=float("inf"),
            shape=(1,) + (h, w),
            dtype=np.float32,
        )

        self.transform = T.Compose([T.Grayscale()])

    def observation(self, observation):

        observation = self.transform(observation)
        assert observation.shape == self.observation_space.shape
        return observation


class ResizeObservation(gym.ObservationWrapper):
    def __init__(self, env: gym.Env, _shape: Union[int, Tuple[int, int]]):
        super().__init__(env)

        self.env = env

        if isinstance(_shape, int):
            shape = (_shape, _shape)
        else:
            shape = _shape

        # self.obs_shape = self.observation_space.shape[0:1] + shape
        (c, _, _) = env.observation_space.shape

        self.observation_space = Box(
            low=-float("inf"), high=float("inf"), shape=(c,) + shape, dtype=np.float32
        )

        self.transforms = T.Compose([T.Resize(shape)])

    def observation(self, observation):
        observation = self.transforms(observation)
        assert observation.shape == self.observation_space.shape
        return observation


class ToTensorObservation(gym.ObservationWrapper):
    def __init__(self, env: gym.Env):
        super().__init__(env)

        # self.obs_shape = env.observation_space.shape[:2]
        (h, w, c) = env.observation_space.shape
        self.observation_space = Box(
            low=-float("inf"),
            high=float("inf"),
            shape=(c, h, w),
            dtype=np.float32,
        )

        self.transform = T.Compose(
            [T.ToTensor(), T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
        )

        # self.transform = T.Compose(
        #     [
        #         T.ToTensor(),
        #         T.Normalize((0.5,), (0.5,)),
        #     ]
        # )

    def observation(self, observation):
        observation = self.transform(observation)
        assert observation.shape == self.observation_space.shape
        return observation


In [10]:
# env = SkipFrame(env, skip=4)
env = ToTensorObservation(env)
env = GrayScaleObservation(env)
env = ResizeObservation(env, 84)
env = FrameStack(env, num_stack=4)
env


<FrameStack<ResizeObservation<GrayScaleObservation<ToTensorObservation<TimeLimit<OrderEnforcing<AtariEnv<BoxingDeterministic-v4>>>>>>>>

In [11]:
# shape is (210, 160, 3)
Observation = npt.NDArray[np.uint8]
Action = int

# shape is (4, 210, 160, 3)
State = torch.Tensor
Reward = int

Transition = TransitionGeneric[State, Action]

In [12]:
o = env.reset()
(o.shape,o.dtype, o[0])

((4, 1, 84, 84),
 torch.float32,
 tensor([[[-2.0337, -2.0337, -2.0337,  ...,  0.2636,  0.2636,  0.2636],
          [-2.0337, -2.0337, -2.0337,  ...,  0.2636,  0.2636,  0.2636],
          [-2.0337, -2.0337, -2.0337,  ...,  0.2636,  0.2636,  0.2636],
          ...,
          [-2.0337, -2.0337, -2.0337,  ...,  0.2636,  0.2636,  0.2636],
          [-2.0337, -2.0337, -2.0337,  ...,  0.2636,  0.2636,  0.2636],
          [-2.0337, -2.0337, -2.0337,  ...,  0.2636,  0.2636,  0.2636]]]))

In [13]:
plt.imshow(o[0], interpolation='nearest')
plt.show()
# out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (720, 1080))
# for frame in frames:
#     out.write(frame) # frame is a numpy.ndarray with shape (1280, 720, 3)
# out.release()

TypeError: Invalid shape (1, 84, 84) for image data

In [ ]:
EVALUATION_TIMES = 30
rwds: List[float] = []

obs: List[Any] = []
for _ in range(1):

    end = False

    rwd = 0
    while not end:
        (o, r, end, _) = env.step(env.action_space.sample())
        obs.append(o)
        # i += 1
        # env.render()
        if r != 0:
            print(r)
        rwd += r
        if end:
            print(f"at end: {r}")

        # if end:
        #     rwds.append(np.sum([r if r is not None else 0 for (_,
        #                                                        _, r) in cast(Episode, episode)]))
    rwds.append(rwd)


1.0
-2.0
1.0
1.0
1.0
-1.0
-1.0
-1.0
1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
-1.0
1.0
1.0
-2.0
-1.0
1.0
1.0
1.0
-1.0
1.0
1.0
1.0
1.0
1.0
-1.0
-1.0
2.0
1.0
-1.0
1.0
2.0
2.0
2.0
-1.0
-1.0
1.0
1.0
1.0
-1.0
1.0
1.0
-1.0
1.0
1.0
-2.0
1.0
-1.0
1.0
1.0
-1.0
1.0
at end: 0.0


In [ ]:
obs[0].frame()

AttributeError: 'Tensor' object has no attribute 'frame'

In [ ]:
rwds

[-21.0]

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=[i + 1 for i in range(len(rwds))],
               y = [r for r in rwds])
)
# fig.update_yaxes(type="log")
# fig.update_layout(yaxis_type="log")
fig.show()

